In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import tensorflow as tf
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from sklearn.model_selection import train_test_split
print(tf.__version__)


2.11.0


In [4]:
train_df=pd.read_csv('train.csv').fillna(' ')
test_df=pd.read_csv('test.csv').fillna(' ')
train_df.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
159570,fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0


In [5]:
x=train_df['comment_text'].values
x

array(["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
       "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)",
       "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",
       ...,
       'Spitzer \n\nUmm, theres no actual article for prostitution ring.  - Crunch Captain.',
       'And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.',
       '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  What kind of communit

In [6]:
train_df.loc[train_df['toxic']==1]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0
...,...,...,...,...,...,...,...,...
159494,fef4cf7ba0012866,"""\n\n our previous conversation \n\nyou fuckin...",1,0,1,0,1,1
159514,ff39a2895fc3b40e,YOU ARE A MISCHIEVIOUS PUBIC HAIR,1,0,0,0,1,0
159541,ffa33d3122b599d6,Your absurd edits \n\nYour absurd edits on gre...,1,0,1,0,1,0
159546,ffb47123b2d82762,"""\n\nHey listen don't you ever!!!! Delete my e...",1,0,0,0,1,0


In [7]:
y=train_df['toxic'].values
y

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [8]:
train_df['toxic'].value_counts()

0    144277
1     15294
Name: toxic, dtype: int64

In [9]:
max_features=20000
max_text_length=400

In [10]:
x_tokenizer=text.Tokenizer(max_features)
x_tokenizer.fit_on_texts(list(x))
x_tokenized=x_tokenizer.texts_to_sequences(x)
x_train_val=sequence.pad_sequences(x_tokenized,maxlen=max_text_length)

In [11]:
x_train_val

array([[    0,     0,     0, ...,  4583,  2273,   985],
       [    0,     0,     0, ...,   589,  8377,   182],
       [    0,     0,     0, ...,     1,   737,   468],
       ...,
       [    0,     0,     0, ...,  3509, 13675,  4528],
       [    0,     0,     0, ...,   151,    34,    11],
       [    0,     0,     0, ...,  1627,  2056,    88]])

In [16]:
embedding_dim=100
embedding_index=dict()
f=open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32') 
    embedding_index[word]=coefs
    
f.close()
print(f'Found {len(embedding_index)} word vectors')

Found 400000 word vectors


In [17]:
embedding_matrix=np.zeros((max_features,embedding_dim))
for word,index in x_tokenizer.word_index.items():
    if index>max_features-1:
        break
    else:
        embedding_vector=embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index]=embedding_vector

In [18]:
model=Sequential()
model.add(Embedding(max_features,
                   embedding_dim,
                   embeddings_initializer=tf.keras.initializers.Constant(
                   embedding_matrix),
                   trainable=False))
model.add(Dropout(0.2))

In [19]:
filters=250
kernel_size=3
hidden_dims=250

In [20]:
model.add(Conv1D(filters,
                kernel_size,
                padding='valid'))
model.add(MaxPooling1D())
model.add(Conv1D(filters,
                5,
                padding='valid',
                activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_dims,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         2000000   
                                                                 
 dropout (Dropout)           (None, None, 100)         0         
                                                                 
 conv1d (Conv1D)             (None, None, 250)         75250     
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 250)        0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, None, 250)         312750    
                                                                 
 global_max_pooling1d (Globa  (None, 250)              0         
 lMaxPooling1D)                                         

In [21]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [22]:
x_train,x_val,y_train,y_val=train_test_split(x_train_val,y,test_size=0.2,random_state=1)


In [23]:
batch_size=32
epochs=3
model.fit(x_train,y_train,
         batch_size=batch_size,
         epochs=epochs,
         validation_data=(x_val,y_val))

Epoch 1/3
3990/3990 [==============================] - 330s 83ms/step - loss: 0.1523 - accuracy: 0.9447 - val_loss: 0.1273 - val_accuracy: 0.9534
Epoch 2/3
3990/3990 [==============================] - 339s 85ms/step - loss: 0.1330 - accuracy: 0.9517 - val_loss: 0.1237 - val_accuracy: 0.9541
Epoch 3/3
3990/3990 [==============================] - 339s 85ms/step - loss: 0.1238 - accuracy: 0.9541 - val_loss: 0.1296 - val_accuracy: 0.9541


In [24]:
test_df=pd.read_csv('test.csv')
test_df

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."


In [25]:
x_test=test_df['comment_text'].values

In [26]:
x_test_tokenized=x_tokenizer.texts_to_sequences(x_test)
x_testing=sequence.pad_sequences(x_test_tokenized,maxlen=max_text_length)

In [27]:
y_testing=model.predict(x_testing,verbose=1,batch_size=32)

4787/4787 [==============================] - 98s 20ms/step


In [28]:
y_testing.shape

(153164, 1)

In [29]:
y_testing[0]

array([0.9937824], dtype=float32)

In [30]:
test_df['Toxic']=['Not Toxic' if x<0.5 else 'Toxic' for x in y_testing]
test_df[['comment_text','Toxic']].head(20)

,comment_text,Toxic
0,Yo bitch Ja Rule is more succesful then you'll...,Toxic
1,== From RfC == \n\n The title is fine as it is...,Not Toxic
2,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",Not Toxic
3,":If you have a look back at the source, the in...",Not Toxic
4,I don't anonymously edit articles at all.,Not Toxic
5,Thank you for understanding. I think very high...,Not Toxic
6,Please do not add nonsense to Wikipedia. Such ...,Not Toxic
7,:Dear god this site is horrible.,Not Toxic
8,""" \n Only a fool can believe in such numbers. ...",Not Toxic
9,== Double Redirects == \n\n When fixing double...,Not Toxic


In [31]:
y_truth=pd.read_csv('test_labels.csv')
y_truth.head(20)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1
5,0001ea8717f6de06,0,0,0,0,0,0
6,00024115d4cbde0f,-1,-1,-1,-1,-1,-1
7,000247e83dcc1211,0,0,0,0,0,0
8,00025358d4737918,-1,-1,-1,-1,-1,-1
9,00026d1092fe71cc,-1,-1,-1,-1,-1,-1


In [32]:
y_test=y_truth.loc[y_truth['toxic']!=-1]
test_df=test_df.loc[y_truth['toxic']!=-1]
print(y_test.shape,test_df.shape)

(63978, 7) (63978, 3)


In [33]:
y_test.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,0,0,0,0,0,0
7,000247e83dcc1211,0,0,0,0,0,0
11,0002f87b16116a7f,0,0,0,0,0,0
13,0003e1cccfd5a40a,0,0,0,0,0,0
14,00059ace3e3e9a53,0,0,0,0,0,0


In [34]:
test_df.head()

,id,comment_text,Toxic
5,0001ea8717f6de06,Thank you for understanding. I think very high...,Not Toxic
7,000247e83dcc1211,:Dear god this site is horrible.,Not Toxic
11,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",Not Toxic
13,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",Not Toxic
14,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",Not Toxic


In [35]:
y_test=y_test['toxic']
y_test.head()

5     0
7     0
11    0
13    0
14    0
Name: toxic, dtype: int64

In [36]:
test_df['Toxic']=[1 if x=="Toxic" else 0 for x in test_df['Toxic']]
test_df=test_df['Toxic']
test_df.head()

C:\Users\Om\AppData\Local\Temp\ipykernel_13860\2165620121.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Toxic']=[1 if x=="Toxic" else 0 for x in test_df['Toxic']]


5     0
7     0
11    0
13    0
14    0
Name: Toxic, dtype: int64

In [37]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
print('Test Data Accuracy Score',accuracy_score(y_test,test_df)*100)
#confusion_matrix(y_test,test_df)
print('Test Data F1 Score ',f1_score(y_test,test_df))

Test Data Accuracy Score 93.07261871268248
Test Data F1 Score  0.6446440025657473


In [49]:
# Replace 'your_input_text' with the text you want to predict
input_text = "i dont think you hate me"

# Tokenize the input text
text_sequence = x_tokenizer.texts_to_sequences([input_text])
# Pad sequences
text_sequence_padded = sequence.pad_sequences(text_sequence, maxlen=max_text_length)
# Make prediction
prediction = model.predict(text_sequence_padded)
# Convert prediction to binary label (0 or 1)
toxicity_label = 1 if prediction[0] >= 0.5 else 0

# Print the result
print("Toxicity Label:", toxicity_label)
print("Prediction Probability:", prediction[0])

1/1 [==============================] - 0s 21ms/step
Toxicity Label: 1
Prediction Probability: [0.64373654]


In [50]:
model.save("toxicity_label_model.h5")

In [51]:
import pickle

# Assuming x_tokenizer is your tokenizer object
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(x_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)